<a href="https://colab.research.google.com/github/archyyu/encoder-related/blob/main/encoder_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import math
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/archyyu/publicResource/main/chat_dataset.csv')

In [3]:
# Hyperparameters
hidden_size = 100
embedding_dim = 40
seq_length = 25
learning_rate = 1e-1
batch_size = 20
dropout = 0.1
eval_iters = 200
num_heads = 8
head_size = 16

In [43]:
pad = '[pad]'
mask = '[MASK]'
data = []
targets = []
for index, row in df.iterrows():
  data.append(row['message'])
  targets.append(row['sentiment'])

In [46]:
datalen = []
for line in data:
  datalen.append(len(line.split(' ')))

In [47]:
targetset = sorted(set(targets))
sentiment_to_index = {s:i for i, s in enumerate(targetset)}
index_to_sentiment = {i:s for i, s in enumerate(targetset)}

In [48]:
dataset = sorted(set((' '.join(data)).split(' ')))
dataset.append(pad)
dataset.append(mask)
vocab_size = len(dataset)
word_to_index = {w:i for i, w in enumerate(dataset)}
index_to_word = {i:w for i, w in enumerate(dataset)}

In [49]:
vocab_size

902

In [53]:
lines = []
for item in data:
  lines.append(item.split(' '))

max_line = max([len(line) for line in lines])

In [54]:
for item in lines:
  for _ in range(max_line - len(item)):
    item.append(pad)

In [55]:
X = []
for line in lines:
  item = [word_to_index[word] for word in line]
  X.append(item)

Y = []
for i in range(len(targets)):
  item = sentiment_to_index[targets[i]]
  Y.append(item)

In [56]:
class AttentionHead(nn.Module):
  def __init__(self, embedding_size, head_size):
    super(AttentionHead, self).__init__()
    self.head_size = head_size
    self.C = embedding_size

    self.q = nn.Linear(self.C, head_size, bias=False)
    self.v = nn.Linear(self.C, head_size, bias=False)
    self.k = nn.Linear(self.C, head_size, bias=False)

  def forward(self, x):
    B,T,C = x.shape
    q = self.q(x)
    k = self.k(x)
    v = self.v(x)

    wei = q @ k.transpose(-2, -1) * (self.head_size ** -0.5)
    wei = F.softmax(wei, dim=-1)

    return wei @ v

class EncoderMultiHeadAttention(nn.Module):
  def __init__(self, num_heads, embedding_size, head_size):
    super(EncoderMultiHeadAttention, self).__init__()
    self.num_heads = num_heads

    self.heads = nn.ModuleList([
        AttentionHead(embedding_size, head_size) for _ in range(num_heads)
    ])

    self.final_linear = nn.Linear(num_heads * head_size, embedding_size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    head_outputs = [head(x) for head in self.heads]
    concatenated_output = torch.cat(head_outputs, dim=-1)
    final_output = self.final_linear(concatenated_output)
    return self.dropout(final_output)

class FeedForward(nn.Module):
  def __init__(self, embedding_size):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(embedding_size, 4 * embedding_size),
        nn.ReLU(),
        nn.Linear(4 * embedding_size, embedding_size),
        nn.Dropout(dropout),
    )
  def forward(self, x):
    return self.net(x)

class EncoderBlockAttention(nn.Module):
  def __init__(self, num_heads, embedding_size, head_size):
    super(EncoderBlockAttention, self).__init__()
    self.multiheads = EncoderMultiHeadAttention(num_heads, embedding_size, head_size)
    self.fw = FeedForward(embedding_size)
    self.norm1 = nn.LayerNorm(embedding_size)
    self.norm2 = nn.LayerNorm(embedding_size)

  def forward(self, x):
    inter_result = x + self.multiheads(self.norm1(x))
    final_result = x + self.fw(self.norm2(inter_result))
    return final_result

class Encoder(nn.Module):
  def __init__(self, num_heads, vocab_size, embedding_size, head_size):
    super(Encoder, self).__init__()
    self.em = nn.Embedding(vocab_size, embedding_size)
    self.pos_encode = nn.Embedding(seq_length, embedding_size)
    self.blocks = nn.ModuleList([EncoderBlockAttention(num_heads, embedding_size, head_size) for _ in range(4)])
    self.f_norm = nn.LayerNorm(embedding_size)
    self.fw = nn.Linear(embedding_size, vocab_size, bias=False)

  def forward(self, x):
    B,T = x.shape
    x_em = self.em(x)
    p_em = self.pos_encode(torch.arange(T))

    x = x_em + p_em
    for block in self.blocks:
      x = block(x)
    x = self.f_norm(x)
    x = self.fw(x)
    return x

criterion = nn.CrossEntropyLoss()

model = Encoder(num_heads, vocab_size, embedding_dim, head_size)
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)

In [107]:
def get_batch():
  inputs = []
  targets = []
  mask_index_list = []
  n = torch.randint(len(X) - batch_size, [1]).item()
  for i in range(batch_size):
    inputs_item = torch.tensor(X[n + i])

    mask_index = torch.randint(datalen[n + 1], [1])

    targets.append(inputs_item[mask_index])
    mask_index_list.append(torch.tensor([i,mask_index]))

    inputs_item[mask_index] = word_to_index[mask]

    inputs.append(inputs_item)

  return torch.stack(inputs), torch.stack(targets), torch.stack(mask_index_list)

In [ ]:
inputs, targets, mask_indices = get_batch()
print(inputs)
print(targets)
print(inputs[mask_indices[:,0],mask_indices[:,1]])

In [118]:
n_iters = 5000
for i in range(n_iters):
  inputs, targets, indices = get_batch()

  predicts = model(inputs)

  #print(targets.view(-1).shape)
  outputs = predicts[indices[:,0],indices[:,1]]
  #print(outputs.shape)

  optimizer.zero_grad(set_to_none=True)

  loss = criterion(outputs, targets.view(-1))

  loss.backward()

  optimizer.step()

  if i%500 == 0:
    print(f'i {i}, loss:{loss.item()}')

i 0, loss:0.857141375541687
i 500, loss:0.6623756885528564
i 1000, loss:2.761378526687622
i 1500, loss:3.16829776763916
i 2000, loss:1.24629807472229
i 2500, loss:0.34113749861717224
i 3000, loss:1.2097088098526
i 3500, loss:2.2975268363952637
i 4000, loss:2.341250419616699
i 4500, loss:2.9368631839752197


In [129]:
hhh = "I have no opinion [MASK] this"

hgg = [word_to_index[word] for word in hhh.split(' ')]
for i in range(max_line - len(hgg)):
  hgg.append(word_to_index[pad])

# hgg = torch.stack(hgg)

hgg = torch.tensor(hgg).unsqueeze(0)

predict = model(hgg)

In [139]:
values, indexes = torch.topk(predict[0][4],2)
for item in indexes:
  print(index_to_word[item.item()])

on
about


that is amazing
I masked one word in the sentence, then training to model to predict the masked word

after some epoches, it works.

will continue